In [11]:
using MotionCaptureJointCalibration
using RigidBodyDynamics
using StaticArrays
using ValkyrieRobot
using ForwardDiff
using Ipopt

Create a `Mechanism` (we'll use the NASA Valkyrie robot) and `MechanismState`:

In [12]:
T = Float64
val = Valkyrie()
mechanism = val.mechanism
remove_fixed_tree_joints!(mechanism)
state = MechanismState{T}(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

Let's generate a random calibration problem, as well as its ground truth solution:

In [13]:
using MotionCaptureJointCalibration.SyntheticDataGeneration
bodies_with_markers = findbody.(mechanism, ["leftFoot", "pelvis"])
srand(1)
body_weights = Dict(b => rand() for b in bodies_with_markers)
problem, groundtruth = generate_calibration_problem(state, body_weights);

In [14]:
problem

CalibrationProblem{Float64} with:
* 25 poses
* 6 calibration parameters
* 8 markers attached to 2 bodies



In [15]:
groundtruth

CalibrationResult{Float64}: Optimal, residual = 0.0. Calibration parameters:
leftAnkleRoll: [0.00473574]
leftAnklePitch: [0.00907607]
leftKneePitch: [-0.00809629]
leftHipRoll: [-0.00972919]
leftHipPitch: [0.000393507]
leftHipYaw: [-0.00393202]

Solve the calibration problem (without using the ground truth result):

In [16]:
solver = IpoptSolver(print_level = 0, max_iter = 10000, derivative_test = "first-order", check_derivatives_for_naninf = "yes", tol = 1e-10)
result = solve(problem, solver)

CalibrationResult{Float64}: Optimal, residual = 1.146371767679625e-9. Calibration parameters:
leftAnkleRoll: [0.00454503]
leftAnklePitch: [0.0088235]
leftKneePitch: [-0.00809516]
leftHipRoll: [-0.00977607]
leftHipPitch: [0.000412891]
leftHipYaw: [-0.00403672]

And visualize the results:

In [17]:
using RigidBodyTreeInspector
using DrakeVisualizer

In [18]:
#NBSKIP
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));

true

In [19]:
vis = Visualizer()[:valkyrie]
setgeometry!(vis, mechanism, parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]))
state = MechanismState{T}(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [20]:
inspect!(state, vis, problem, result)

Interact.Options{:SelectionSlider,Bool}(5: "input-3" = false Bool , "", false, "Before cal", Interact.OptionDict(DataStructures.OrderedDict("Before cal"=>false,"After cal"=>true), Dict(false=>"Before cal",true=>"After cal")), Any[], Any[], true, "horizontal")

Interact.Slider{Int64}(7: "input-4" = 1 Int64 , "Pose number", 1, 1:25, "horizontal", true, "d", true)